In [6]:
from pylab import *

from statsmodels.tsa.ar_model import AR # import AR

%matplotlib inline

# Autoregression models

Those are models of the form

$$
X_t = a + \sum_{i=1}^{p}b_iX_{t-i}+\epsilon
$$
where $\epsilon$ is allowed to be stochastic. As such, they should catch all recurrence relations. Maybe even some that are not generated by recurrences. 

In [47]:
# simple example of use

seq = array([1,1,1,1,1,1,1]) 
n = size(seq)
print n

model = AR(seq)
result= model.fit(5,ic='AIC',disp=True)

# still trying to understand how the prediction function
# works
result.predict(start=5,end=6)

7


array([ 1.,  1.])

In [48]:
# Bring in the data

testfile='../data/train.csv'
data = open(testfile).readlines()

# This part of the code was stolen from kaggle
sequences = {}
for i in range(1,1001):
    line=data[i]
    line =line.replace('"','')
    line = line[:-1].split(',')
    id = int(line[0])
    sequence=[int(x) for x in line[1:]];
    sequences[i]=sequence # throwing away the ids

print len(sequences)



1000


In [52]:
def guessAR(seqA):
    # Guessing a bunch of sequences stored in the dictionary
    # seqA. 
    # 
    n = len(seqA)
    preds = zeros(n)
    errors= ones(n)
    for i in xrange(1,n):
        seq = seqA[i]
        seq_size = size(seq)
        
        if seq_size<=4:
            # skipping small sequences
            # for now
            continue
            
        model = AR(seq)
        result = model.fit(maxlag=(seq_size-1),ic='bic',disp=True)
        
        try:
            preds[i] = round(result.predict(seq_size,seq_size))
        except ValueError:
            print "Error for seq.", seq
            print "at position i=",i
            
        errors[i] = abs(round(result.predict(seq_size-1,seq_size-1))-seq[seq_size-1])
        
    return preds, errors
    

In [53]:
preds, errors = guessAR(sequences)

# number below is smaller than optimal
# 
print 'guessed right',round(sum(errors<1e-1)*1.0/len(sequences)*100.0),
print "% out of", len(sequences)

guessed right 11.0 % out of 1000
